# Transfer Learning

## <center> Introduction to transfer learning

### what is it?

Transfer learning is a training method that use 'source data' that is not directly relevant to your task to train your model.

### type overview

| - | - |source data | source data 
| :---: | :---: | :---: | :---: |
| - | -  | labelled | unlabelld |
| Target data| labelled | Model Fine-tuning|
|Target data | unlabelled

label-label:

* Target data:($x^t$, $y^t$) few
* Source data:($x^s$, $y^s$) a large amount

label-unlabelled:

* Target data:($x^t$) as our testing data
* Source data:($x^s$, $y^s$) as our training data

## <center> Model Fine-tuning (label - label)

### Introduction

how to train:

training your model using source data, then fine-tune the model on target data.

Issue:

Only has few target data, model might be easily overfitting. 

### Type1: Conservative Training

**intro**

use parameters in the model trained by target data as initialisation. When you do fine-tune on your target data, add some constarint like "for the same input, the output of two model(trained by target data and fine-tuned model) should be similar."

### Type2: Layer Transfer

**intro**

fix some layers in original model(keep the paramters) and only update or tune remaining layers.

It can prevent overfitting since only update few parameters.

You can fine-tune whole layers when have sufficient data.

**which layers should be transferred(fixed)**

Generally
* Image: transfer the first few layers. (The first few layers normally recognise some global features)
* Audio: transfer the last few layers. (The first few layers in a speech recognised NN often analyse the manner of articulation of speakers and remaining layers conclude the content.)

**performance on image**

some research shows that with more layers transferred, the performance on 'transfer + fine-tune' is higher than 'base line(only train model on target data), and 'base line' is higher than 'transfer'

## <center> Multitask Learning(label-label)

### introduction

Multitask Learning uses a multi-layer NN structure. 

That is, from some one hidden layer, it generates several branches hidden layers and these branches still connect their own layers. Each branch has their own output layer which complete different tasks.

Or there are several input features and conneted to their own hidden layers, then they come to some common layers then seperate again.

### Progressive Neural Networks

construct a NN a to complete task 1.

construct a NN b to complete task 2, but the input each layer in b is the output from previous layer in NN b and the output from a layer in a.

construct a NN c to complete task3, but the input each layer in b is the output from previous layer in NN c, the output from a layer in b and the output from a layer in a.

...

## <center> Domain-adversarial training

### introduction

**Data:**

Target data and source data should come from different domain.

**Domain:**

target data or source data

**structure:**

we construct a NN take target data and source data as input, then output a feature vector. This NN is called feature extractor, it aims to eliminate the domain feature.

Moreover, we construct a NN take the feature vector as input, then output a domain label. This NN is called Domain classifer, it aims to distinguish the domain of feature vector.

Finally, we construct a NN also take the feature vector as input, then output class label for classification. This NN is called Label predictor, it aims to do classifcation.

This structure is similar to GAN, feature extractor should try best to cheat Domain classifier and Domain classifier should struggle to distinguish data domain.

### How to train

We train these three NNs as a whole NN. 

However, we need to do a little adjust when we do gradient descent. That is, the gradient of feature extractor and the gradient of label predictor are in positive correlation because we want to predict class correctly. 

Nevertheless,  the gradient of feature extractor and the gradient of domain classifier are in negative correlation because if we update parameters in feature extractor to increase gradient in classifier, that could incrase the loss of feature extractor. Hence we will cheat it finally.

To achieve this, an additional layer called 'gradient reversal layer' should be added between the output of feature extractor and the input of domain classifier. It will reverse the gradient computed in domain classifer when do propagation.


## <center> Zero-shot Learning 

### introduction

**Data:**

Source data and target data should work for different tasks.

For example, the label of Source data is set A, the label of target data is set B. Then $A \cap B = \emptyset$

**How to train**

Generally, we represent each class by a vector of attributes.

There are two method to do it.

### Database of attributes

Firstly, we need create a database. The index of database is different label and the columns of database is a series of attributes. 

The number of attributes should be sufficient to make each label has unique attributes tuple. That is, if $Tuple(A) = Tuple(B)$, then $A = B$, if $Tuple(A) \ne Tuple(B)$, then $A \ne B$.

Secondly, Let's train a NN take inputs and output a vector(Tuple) of attributes. This vecotr(Tuple) could hepl us find which label it should belongs to or be hte most similar to by scan database.



### Attribute embedding + word embedding

**structure**

Similar to last method, we need to train a NN take inputs and output a vector of attributes.

However, since there is no database for match checking, we can use the technique of word embedding to project a bunch of words to the same space as attributes. 

Then the word vector is the label of a data with the greatest similarity between word vector and attributes vector.

**Loss function of zero-shot**

if we define 

$$Loss = argmin_{f, g} \sum_{n}|| f(x^n) - g(y^n)||_2$$

, where f(x) give us attributes vector and g(y) give us word embedding of label. It hope to find $f$ and $g$ such that every data and their label has the minimum distance in space. 

The problem in here is loss could reduce to 0 if whatever input $f$ and $g$ takes, they always output same results.

Hence, Loss function should be modified to 

$$
Loss = argmin_{f, g}\sum_n max(0, k-f(x^n) \cdot g(y^n) + max_{m \ne n} f(x^n) \cdot g(y^m))
$$

Let's focus on 
$$
k-f(x^n) \cdot g(y^n) + max_{m \ne n} f(x^n) \cdot g(y^m)
$$

Our goal is to minimise this part as much as possible (less than 0) to decrease loss to 0.

That is 

$$
k-f(x^n) \cdot g(y^n) + max_{m \ne n} f(x^n) \cdot g(y^m) < 0
$$

also is 

$$
f(x^n) \cdot g(y^n) - max_{m \ne n} f(x^n) \cdot g(y^m) > k
$$

this means we hope the similarity between a data and its real label to be as great as possible. At the same time, the similarity between this data and any other false label to be as less as possible.

more make sense.

### convex combination of semantic embedding

Given a well-trained NN, it takes our data as input, then output a vecotr $P$ which is the probability vector. This vector tell us probability of this data belongs to each class(label).

Then given a well-trained word embedding database, we find the vector has the greast similarity with $\sum_{n} p^n V^n$, where $p^n$ is nth element in $P$, and $V^n$ is the corresponding label in word embedding. This vector with the greast similarity is the predicted label.

### Application of zero-shot learning

Google Neural Machine Translation